In [14]:
import spacy
import json
import random

def load_data(file):
    data = []
    with open(file, 'r', encoding="latin1") as f:
        for line in f:
            line=json.loads(line)
            if "label" in line:
                line["entities"] = line.pop("label")
            else:
                line["entities"] = []
            tmp_ents = []
            for e in line["entities"]:
                if e[2] in ['SKILL', 'SOFT-SKILL']:
                    tmp_ents.append((e[0], e[1], e[2]))

                line["entities"] = tmp_ents
                data.append((line["data"], line["entities"]))
    return data

def save_data(file, data):
    with open (file, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent =4)

In [15]:
from spacy.tokens import DocBin
def spacySave(training_data, train=True, test=False):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annotations in training_data:
        doc = nlp(text)
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    if(train):
        db.to_disk("./data/train.spacy")
    if(test):
        db.to_disk("./data/test.spacy")

In [16]:
TRAIN_DATA = load_data("data/train.jsonl")
spacySave(TRAIN_DATA)

In [17]:
TEST_DATA = load_data("data/test.jsonl")
spacySave(TEST_DATA, train=False, test=True)

In [18]:
from spacy.cli.train import train
train("./config.cfg", "./output", overrides={"paths.train": "./data/train.spacy", "paths.dev": "./data/test.spacy"})

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================


KeyboardInterrupt: 

# THIS DID NOT WORK

In [ ]:
def train_spacy(TRAIN_DATA, iterations):
    nlp = spacy.blank("en")
    #ruler = nlp.add_pipe("entity_ruler")
    #skill_pattern_path = "jz_skill_patterns.jsonl"
    #ruler.from_disk(skill_pattern_path)
    ner = nlp.add_pipe("ner", name="skills_ner")
    ner.add_label("SKILL")
    ner.add_label("SOFT-SKILL")
    other_pipes = [pipes for pipe in nlp.pipe_names if pipe !="skills_ner"]
    with nlp.disable_pipes(other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print(f"Starting iteration {(str(itn))}")
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                print(text)
                nlp.update([text],
                           [annotations], 
                           drop=0.2, 
                           sgd=optimizer, 
                           losses=losses
                )
            print(losses)
    return(nlp)
                